In [ ]:
# default_exp cli

In [ ]:
#export
from pathlib import Path
from tightai.projects import Project, Version
from tightai.local import local_server

In [ ]:
#export
import argparse

SERVICE_OPTIONS = [
    "auth",
    "local",
    "project",
    "projects",
    "version",
    "versions"
]

ACTION_OPTIONS = ["get", "create", "delete", "run"]

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("service", type=str, help="service", choices=SERVICE_OPTIONS)
    parser.add_argument("action", type=str, help="Action for service", nargs='?', choices=ACTION_OPTIONS)
    parser.add_argument("--all", help='Flag to view all results', type=str)
    parser.add_argument("-p", "--project", help='Use a valid `project_id`', type=str)
    parser.add_argument("-v", "--version", type=int)
    parser.add_argument("-d", "--dir", help="Pahth to local directory. (Same as `--path`)", type=str)
    parser.add_argument("--path", help="Path to local directory. (Same as `--dir`)", type=str)
    parser.add_argument("--port", help="Use a valid localhost port.", type=int, default=5007)
    parser.add_argument('--name', help="Used in creating items in services like Projects", type=str)
    parser.add_argument("--push", help="Flag to deploy a project version.", action='store_true')
    parser.add_argument("--status", help="Refresh status", action='store_true')
    parser.add_argument("--destroy", help="Takes version offline.\nTo delete forever, use: `tightai version delete -p <project_id> -v <version>`\n", action='store_true')
    parser.add_argument("--open", help="Opens the url for a deployed version.", action='store_true')
    
    
    args = parser.parse_args()
    # print(args.service, args.action, args.version, args.dir, args.project)
    
    service = args.service
    action = args.action or "get"
    project_id = args.project or None
    version = args.version or None
    name = args.name or None
    push = args.push or False
    path = args.dir or None
    status = args.status or False
    destroy = args.destroy or False
    open_url = args.open or False
    if path == None:
        path = args.path or None
    
    if service == "local":
        port = args.port or 5007
        path = path or "."
        print(f"Running local tight.ai server on {path} with port {port}")
        local_server(path=path, port=port)
    elif service == "project" or service == "projects":
        if action == "get":
            if project_id != None and f"{project_id}".strip() != "":
                project = Project.objects.get(project_id=project_id)
                project.details() # prints project details
            else:
                projects = Project.objects.all()
                for p in projects:
                    p.summary(verbose=1)
                    print('\n')
        elif action == "create":
            new_project = None
            if name != None and project_id == None:
                new_project = Project.objects.create(name=name)
            elif name != None and project_id != None:
                new_project = Project.objects.create(name=name, id=project_id)
            elif name == None and project_id != None:
                new_project = Project.objects.create(id=name)
            if new_project:
                new_project.details()
                print(project_id, "created successfully")
    elif service == 'version' or service == 'versions':
            if project_id == None:
                    raise Exception("You must include a Project ID when looking up a version. Include `-p <project_id>`.")
            if action == "get" and push == False: 
                if version != None:
                    version_obj = Version.objects.get(project_id=project_id, version=version)
                    if status == True and destroy == False:
                        version_obj.status()
                    elif status == False and destroy == True:
                        version_obj.destroy()
                    else:
                        details = version_obj.details()
                        if open_url:
                            try:
                                version_obj.open()
                            except Exception as e:
                                print('\n', e)
                    print('\n')
                else:
                    print(f"{project_id} versions:")
                    project = Project.objects.get(project_id=project_id)
                    versions = project.get_versions()
                    for v in versions:
                        v.summary(verbose=1)
                        print('\n')
            elif action == "create" and push == False:
                new_version = None
                created = False
                if version == None:
                    raise Exception("You must specify a version number")
                else:
                    new_version, created = Version.objects.get_or_create(version=version, project_id=project_id)
                    if not created:
                        print(f"Version {version} already exists for project `{project_id}`.")
                if new_version != None:
                    if created:
                        print(f"Version {version} created succseffuly for project `{project_id}`.")
                    print(new_version.details())
            if action == "delete":
                if version == None:
                    raise Exception("A valid project version is required.")
                version_obj = Version.objects.get(project_id=project_id, version=version)
                version_obj.delete()
            if push == True:
                if path == None:
                    raise Exception("You must include a directory path with you push a verion. Append `--path .` to use your current directory.")
                new_version = None
                created = False
                if version == None:
                    raise Exception("You must specify a version number")
                else:
                    print(f"Pushing {project_id} v{version} from {path}")
                    new_version, created = Version.objects.get_or_create(version=version, project_id=project_id)
                    new_version.push(Path(path))

In [ ]:
#hide
# if __name__=="__main__":
    # main('versions', action='get', project_id='spam-sms', version=1)
    # main('versions', action='get', project_id='spam-sms')
    # main('projects')
    # main('projects', project_id='spam-sms')
    
    # main('projects', action='get', project_id='hello-worlderee')
    # main('versions', action='create', project_id='hello-worlderee', version=1)
    # main('versions', action='get', project_id='hello-worlderee', version=1, ) # path='/Users/jmitch/tight/my-tight-apps/local_dev/')
    # main('versions', action='push', project_id='hello-worlderee', version=1, path='/Users/jmitch/tight/my-tight-apps/local_dev/')
    # main('local', path='/Users/jmitch/tight/my-tight-apps/spam-sms')